In [1]:
import json
import os
from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
import sys

In [2]:
## Loading openai key
with open('../environ.json') as f:
    config = json.load(f)
    api_keys = list(config.values())[0]

# assign openai key to an environment variable
os.environ["OPENAI_API_KEY"] = api_keys

# # check to confirm
# os.getenv("OPENAI_API_KEY")


In [1]:
import json 
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

## Loading openai key
with open('../environ.json') as f:
    config = json.load(f)
    api_keys = list(config.values())[0]

# assign openai key to an environment variable
os.environ["OPENAI_API_KEY"] = api_keys

# # check to confirm
# os.getenv("OPENAI_API_KEY")


In [3]:
# establish connection to neo4j graph database
graph = Neo4jGraph(
    url="bolt://localhost:7689", 
    username="admin", 
    password="cran2graph", 
    database="neo4j"
)
print(graph.schema)

Node properties are the following:
Person {name: STRING},Package {latestPublishedDate: STRING, name: STRING, md5sum: STRING, description: STRING, license: STRING, version: STRING},Organization {name: STRING},License {type: STRING},Institution {type: STRING}
Relationship properties are the following:

The relationships are the following:
(:Person)-[:CONTRIBUTED_TO]->(:Package),(:Person)-[:WORKS_AT]->(:Institution),(:Person)-[:MAINTAINS]->(:Package),(:Package)-[:DEPENDS_ON]->(:Package),(:Package)-[:LICENSE_BY]->(:License),(:Package)-[:SUPPORTED_BY]->(:Organization)


In [6]:
# chain rule
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [13]:
b = chain.invoke('How many packages is maintained by Kirill Müller?')['result']



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Kirill Müller'})-[:MAINTAINS]->(p:Package)
RETURN COUNT(p) as numberOfPackagesMaintainedByKirillMuller;
Full Context:
[{'numberOfPackagesMaintainedByKirillMuller': 28}]

> Finished chain.


In [12]:
a['result'] 

'Kirill Müller maintains 28 packages.'

In [14]:
b

'Kirill Müller maintains 28 packages.'

In [7]:
def evaluate_accuracy(chain, questions, expected_answers):
    num_correct = 0
    total_questions = len(questions)
    
    for i, question in enumerate(questions):
        print(f"---Questions: {question}----")
        generated_answer = chain.invoke(question)['result']
        print(f"KGbased Answer: {generated_answer}")
        expected_answer = expected_answers[i]
        
        if expected_answer in generated_answer:
            num_correct += 1
            print(f"Number Correct: {num_correct}")
    accuracy = num_correct / total_questions
    return accuracy

questions = [
    "How many packages is maintained by Kirill Müller?",
    "How many packages has Kirill Müller contributed to?",
    "Name of package Dennis Irorere contributed to",
    "Who maintains anyflights package",
    "Who is maintaining the most packages?",
    "Who is contributing to the most to packages?",
    'What package have the most contributions',
    "What packages depend on the most?"
]

expected_answers = [
    "28",
    "54",
    "anyflights",
    "Simon P. Couch",
    "Dirk Eddelbuettel",
    "Hadley Wickham",
    "mlpack",
    "stats"
]

In [8]:
accuracy = evaluate_accuracy(chain, questions, expected_answers)
print(f'Accuracy: {accuracy}')

---Questions: How many packages is maintained by Kirill Müller?----


c:\Users\Dee\root\Projects\dev\devGraph\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Kirill Müller'})-[:MAINTAINS]->(p:Package)
RETURN COUNT(p)
Full Context:
[{'COUNT(p)': 28}]

> Finished chain.
KGbased Answer: Kirill Müller maintains 28 packages.
Number Correct: 1
---Questions: How many packages has Kirill Müller contributed to?----


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Kirill Müller'})-[:CONTRIBUTED_TO]->(p:Package)
RETURN COUNT(p) as numberOfPackagesContributed
Full Context:
[{'numberOfPackagesContributed': 54}]

> Finished chain.
KGbased Answer: Kirill Müller has contributed to 54 packages.
Number Correct: 2
---Questions: Name of package Dennis Irorere contributed to----


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Dennis Irorere"})-[:CONTRIBUTED_TO]->(pkg:Package)
RETURN pkg.name
Full Context:
[{'pkg.name': 'anyflights'}]

> Finished chain.
KGbased Answer: The name of the package Dennis Iro

Using gpt3.5 models

In [15]:
from openai import OpenAI

In [16]:
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant. I want to get information about the CRAN package ecosystem"},
    {"role": "user", "content": "How many packages is maintained by Kirill Müller?"}
  ]
)

response.choices[0].message.content

In [ ]:
def generate_answer_gpt(questions):
    response = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "system", "content": "You are a helpful assistant. I want to get information about the CRAN package ecosystem"},
            {"role": "user", "content": questions}
        ]
    )
    
    gpt_answer = response.choices[0].message.content.strip()
    return gpt_answer

def evaluate_accuracy(chain, questions, expected_answers):
    num_correct = 0
    total_questions = len(questions)
    
    for i, question in enumerate(questions):
        print(f"---Questions: {question}----")
        kg_generated_answer = chain.invoke(question)['result']
        gpt_generated_answer = generate_answer_gpt(question)
        print(f"KGbased Answer: {kg_generated_answer}")
        print(f"GPTbased Answer: {gpt_generated_answer}")
        expected_answer = expected_answers[i]
        
        if expected_answer in kg_generated_answer:
            num_correct += 1
            print(f"Number Correct: {num_correct}")
    accuracy = num_correct / total_questions
    return accuracy

questions = [
    "How many packages is maintained by Kirill Müller?",
    "How many packages has Kirill Müller contributed to?",
    "Name of package Dennis Irorere contributed to",
    "Who maintains anyflights package",
    "Who is maintaining the most packages?",
    "Who is contributing to the most to packages?",
    'What package have the most contributions',
    "What packages depend on the most?"
]

expected_answers = [
    "28",
    "54",
    "anyflights",
    "Simon P. Couch",
    "Dirk Eddelbuettel",
    "Hadley Wickham",
    "mlpack",
    "stats"
]

In [22]:
import pandas as pd

In [37]:
def generate_answer_gpt(questions):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. I want to get information about the open source package ecosystem, your answer should be brief and a summary max 70 characters"},
            {"role": "user", "content": questions}
        ]
    )
    
    gpt_answer = response.choices[0].message.content.strip()
    return gpt_answer

def evaluate_accuracy(chain, questions, expected_answers):
    num_correct_kg = 0
    num_correct_gpt = 0
    total_questions = len(questions)
    
    results = []

    for i, question in enumerate(questions):
        kg_generated_answer = chain.invoke(question)['result']
        gpt_generated_answer = generate_answer_gpt(question)
        expected_answer = expected_answers[i]
        
        kg_correct = 1 if expected_answer in kg_generated_answer else 0
        gpt_correct = 1 if expected_answer in gpt_generated_answer else 0

        results.append({
            "Question": question,
            "Expected Answer": expected_answer,
            "KG-based Answer": kg_generated_answer,
            "GPT-based Answer": gpt_generated_answer,
            "KG Correct": kg_correct,
            "GPT Correct": gpt_correct
        })

        num_correct_kg += kg_correct
        num_correct_gpt += gpt_correct

    accuracy_kg = num_correct_kg / total_questions
    accuracy_gpt = num_correct_gpt / total_questions

    return results, accuracy_kg, accuracy_gpt

questions = [
    "How many packages is maintained by Kirill Müller?",
    "How many packages has Kirill Müller contributed to?",
    "Name of package Dennis Irorere contributed to",
    "Who maintains anyflights package",
    "Who is maintaining the most packages?",
    "Who is contributing to the most to packages?",
    'What package have the most contributions',
    "What packages depend on the most?",
    "Who is the creator of the pandas package?"
]

expected_answers = [
    "28",
    "54",
    "anyflights",
    "Simon P. Couch",
    "Dirk Eddelbuettel",
    "Hadley Wickham",
    "mlpack",
    "stats",
    "Wes McKinney"
]

results, accuracy_kg, accuracy_gpt = evaluate_accuracy(chain, questions, expected_answers)

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Calculate total correct for both models
total_correct_kg = df_results["KG Correct"].sum()
total_correct_gpt = df_results["GPT Correct"].sum()

# Calculate accuracy across both models
total_questions = len(questions)
total_accuracy_kg = total_correct_kg / total_questions
total_accuracy_gpt = total_correct_gpt / total_questions

# Display results
print("KG-based Accuracy:", accuracy_kg)
print("GPT-based Accuracy:", accuracy_gpt)
print("\nTotal Correct Answers:")
print("KG-based:", total_correct_kg)
print("GPT-based:", total_correct_gpt)
print("\nTotal Accuracy:")
print("KG-based:", total_accuracy_kg)
print("GPT-based:", total_accuracy_gpt)

# Display results DataFrame
print("\nResults:")
print(df_results)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Kirill Müller"})-[:MAINTAINS]->(pkg:Package)
RETURN COUNT(pkg) as numberOfPackagesMaintainedByKirillMuller;
Full Context:
[{'numberOfPackagesMaintainedByKirillMuller': 28}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Kirill Müller'})-[:CONTRIBUTED_TO]->(p:Package)
RETURN COUNT(p) as numberOfPackagesContributed
Full Context:
[{'numberOfPackagesContributed': 54}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Dennis Irorere"})-[:CONTRIBUTED_TO]->(pkg:Package)
RETURN pkg.name
Full Context:
[{'pkg.name': 'anyflights'}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:MAINTAINS]->(pkg:Package {name: "anyflights"})
RETURN p
Full Context:
[{'p': {'name': 'Simon P. Couch'}}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
G

In [38]:
df_results.to_csv('../data/accuracy_results.csv', index=False)

In [24]:
def generate_answer_gpt(model, questions):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant. I want to get information about the CRAN package ecosystem"},
            {"role": "user", "content": questions}
        ]
    )
    
    gpt_answer = response.choices[0].message.content.strip()
    return gpt_answer

def evaluate_accuracy(chain, questions, expected_answers):
    num_correct_kg = 0
    num_correct_gpt35 = 0
    num_correct_gpt4 = 0
    total_questions = len(questions)
    
    results = []

    for i, question in enumerate(questions):
        kg_generated_answer = chain.invoke(question)['result']
        gpt35_generated_answer = generate_answer_gpt(model='gpt-3.5-turbo', questions=question)
        gpt4_generated_answer = generate_answer_gpt(model='gpt-4-turbo', questions=question)
        expected_answer = expected_answers[i]
        
        kg_correct = 1 if expected_answer in kg_generated_answer else 0
        gpt35_correct = 1 if expected_answer in gpt35_generated_answer else 0
        gpt4_correct = 1 if expected_answer in gpt4_generated_answer else 0

        results.append({
            "Question": question,
            "Expected Answer": expected_answer,
            "KG-based Answer": kg_generated_answer,
            "GPT3.5-based Answer": gpt35_generated_answer,
            "GPT4-based Answer": gpt4_generated_answer,
            "KG Correct": kg_correct,
            "GPT3.5 Correct": gpt35_correct,
            "GPT4 Correct": gpt4_correct,
        })

        num_correct_kg += kg_correct
        num_correct_gpt35 += gpt35_correct
        num_correct_gpt4 += gpt4_correct

    accuracy_kg = num_correct_kg / total_questions
    accuracy_gpt35 = num_correct_gpt35 / total_questions
    accuracy_gpt4 = num_correct_gpt4 / total_questions

    return results, accuracy_kg, accuracy_gpt35, accuracy_gpt4

questions = [
    "How many packages is maintained by Kirill Müller?",
    "How many packages has Kirill Müller contributed to?",
    "Name of package Dennis Irorere contributed to",
    "Who maintains anyflights package",
    "Who is maintaining the most packages?",
    "Who is contributing to the most to packages?",
    'What package have the most contributions',
    "What packages depend on the most?",
    "total package that depends on dplyr",
]

expected_answers = [
    "28",
    "54",
    "anyflights",
    "Simon P. Couch",
    "Dirk Eddelbuettel",
    "Hadley Wickham",
    "mlpack",
    "stats",
    "134"
]

results, accuracy_kg, accuracy_gpt35, accuracy_gpt4 = evaluate_accuracy(chain, questions, expected_answers)

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Calculate total correct for all models
total_correct_kg = df_results["KG Correct"].sum()
total_correct_gpt35 = df_results["GPT3.5 Correct"].sum()
total_correct_gpt4 = df_results["GPT4 Correct"].sum()

# Calculate accuracy across all models
total_questions = len(questions)
total_accuracy_kg = total_correct_kg / total_questions
total_accuracy_gpt35 = total_correct_gpt35 / total_questions
total_accuracy_gpt4 = total_correct_gpt4 / total_questions

# Display results
print("KG-based Accuracy:", accuracy_kg)
print("GPT3.5-based Accuracy:", accuracy_gpt35)
print("GPT4-based Accuracy:", accuracy_gpt4)
print("\nTotal Correct Answers:")
print("KG-based:", total_correct_kg)
print("GPT3.5-based:", total_correct_gpt35)
print("GPT4-based:", total_correct_gpt4)
print("\nTotal Accuracy:")
print("KG-based:", total_accuracy_kg)
print("GPT3.5-based:", total_accuracy_gpt35)
print("GPT4-based:", total_accuracy_gpt4)

# Display results DataFrame
print("\nResults:")
print(df_results)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Kirill Müller"})-[:MAINTAINS]->(pkg:Package)
RETURN COUNT(pkg)
Full Context:
[{'COUNT(pkg)': 28}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Kirill Müller'})-[:CONTRIBUTED_TO]->(p:Package)
RETURN COUNT(p) as total_packages_contributed
Full Context:
[{'total_packages_contributed': 54}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Dennis Irorere"})-[:CONTRIBUTED_TO]->(pkg:Package)
RETURN pkg.name
Full Context:
[{'pkg.name': 'anyflights'}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:MAINTAINS]->(pkg:Package {name: "anyflights"})
RETURN p;
Full Context:
[{'p': {'name': 'Simon P. Couch'}}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:MAINTAINS]->(pkg:Package)
RETURN p.name,

In [ ]:
def generate_answer_gpt(questions):
    response = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "system", "content": "You are a helpful assistant. I want to get information about the CRAN package ecosystem"},
            {"role": "user", "content": questions}
        ]
    )
    
    gpt_answer = response.choices[0].message.content.strip()
    return gpt_answer

In [21]:
response.choices[0].message.content.strip()

'Kirill Müller maintains a total of 22 packages on CRAN as of now.'

In [20]:
response.choices[0].message.content.strip()

'Kirill Müller maintains a total of 22 packages on CRAN as of now.'

Consistency

In [26]:
def evaluate_consistency(chain, questions, num_epochs):
    results = []
    
    for epoch in range(num_epochs):
        epoch_results = []
        
        for question in questions:
            kg_generated_answers = []
            gpt_generated_answers = []
            
            # KG-based approach
            try:
                kg_generated_answer = chain.invoke(question)['result']
                kg_generated_answers.append(kg_generated_answer)
            except:
                kg_generated_answer = "N/A"
                kg_generated_answers.append(kg_generated_answer)
            
            # GPT-based approach
            try:
                gpt_generated_answer = generate_answer_gpt(model='gpt-3.5-turbo', questions=question)
                gpt_generated_answers.append(gpt_generated_answer)
            except:
                gpt_generated_answer = "N/A"
                gpt_generated_answers.append(gpt_generated_answer)
            
            epoch_results.append({
                "Question": question,
                f"KG-based Answer Epoch {epoch+1}": kg_generated_answer,
                f"GPT-based Answer Epoch {epoch+1}": gpt_generated_answer
            })
        
        results.extend(epoch_results)
    
    # Convert results to DataFrame
    df_results = pd.DataFrame(results)
    
    return df_results

# Define questions
questions = [
    "How many packages is maintained by Kirill Müller?",
    "How many packages has Kirill Müller contributed to?",
    "Name of package Dennis Irorere contributed to",
    "Who maintains anyflights package",
    "Who is maintaining the most packages?"
]

# Set number of epochs
num_epochs = 3

# Evaluate consistency
consistency_results = evaluate_consistency(chain, questions, num_epochs)

# Display consistency results
print(consistency_results)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Kirill Müller"})-[:MAINTAINS]->(pkg:Package)
RETURN COUNT(pkg)
Full Context:
[{'COUNT(pkg)': 28}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person{name: "Kirill Müller"})-[:CONTRIBUTED_TO]->(p:Package)
RETURN COUNT(p)
Full Context:
[{'COUNT(p)': 54}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Dennis Irorere"})-[:CONTRIBUTED_TO]->(pkg:Package)
RETURN pkg.name
Full Context:
[{'pkg.name': 'anyflights'}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Package {name: "anyflights"})<-[:MAINTAINS]-(person:Person)
RETURN person;
Full Context:
[{'person': {'name': 'Simon P. Couch'}}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:MAINTAINS]->(pkg:Package)
RETURN p.name, COUNT(pkg) AS numPackages
ORDER BY 

In [32]:
response1 = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant. I want to get information about the open source package ecosystem"},
    {"role": "user", "content": "How ?"}
  ]
)

response1.choices[0].message.content

'The pandas package, a popular open-source data analysis and manipulation tool for Python, was created by Wes McKinney. He started working on pandas in 2008 while in need of high performance, flexible tool for analysis of data.'

In [ ]:

Who is the creator of the pandas package? Wes McKinney

In [7]:
chain.run("How many packages is maintained by Kirill Müller?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Kirill Müller'})-[:MAINTAINS]->(p:Package)
RETURN COUNT(p)
Full Context:
[{'COUNT(p)': 28}]

> Finished chain.


'Kirill Müller maintains 28 packages.'

In [9]:
chain.run("how many packages does Jing Zhang maintains and the names and when it was published?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Jing Zhang'})-[:MAINTAINS]->(pkg:Package)
RETURN COUNT(pkg) AS packageCount, pkg.name AS packageName, pkg.latestPublishedDate AS publishedDate
Full Context:
[{'packageCount': 1, 'packageName': 'ggrisk', 'publishedDate': '2021-08-09'}, {'packageCount': 1, 'packageName': 'do', 'publishedDate': '2021-08-03'}, {'packageCount': 1, 'packageName': 'msig', 'publishedDate': '2021-06-29'}, {'packageCount': 1, 'packageName': 'base.rms', 'publishedDate': '2020-08-01'}, {'packageCount': 1, 'packageName': 'cutoff', 'publishedDate': '2019-12-20'}, {'packageCount': 1, 'packageName': 'nomogramFormula', 'publishedDate': '2020-01-28'}]

> Finished chain.


"Jing Zhang maintains a total of 6 packages. The names of the packages are 'ggrisk', 'do', 'msig', 'base.rms', 'cutoff', and 'nomogramFormula'. Here are the published dates for each package: 'ggrisk' was published on August 9, 2021, 'do' was published on August 3, 2021, 'msig' was published on June 29, 2021, 'base.rms' was published on August 1, 2020, 'cutoff' was published on December 20, 2019, and 'nomogramFormula' was published on January 28, 2020."

In [11]:
chain.run("Name of package Dennis Irorere contributed to")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Dennis Irorere'})-[:CONTRIBUTED_TO]->(p:Package)
RETURN p.name
Full Context:
[{'p.name': 'anyflights'}]

> Finished chain.


'Dennis Irorere contributed to the package named "anyflights."'

In [14]:
chain.run("how many packages does Kirill Müller contributes and list the packages. I might have spelt name wrong")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Kirill Müller'})-[:CONTRIBUTED_TO]->(pkg:Package)
RETURN COUNT(pkg) AS numberOfPackages, COLLECT(pkg.name) AS packageList
Full Context:
[{'numberOfPackages': 54, 'packageList': ['RSQLite', 'here', 'RMariaDB', 'DBI', 'chk', 'roxygen2md', 'mlfit', 'StrucDiv', 'term', 'dm', 'DBItest', 'log4r', 'hms', 'skimr', 'bindrcpp', 'desc', 'indiedown', 'enc', 'knitrProgressBar', 'fledge', 'cli', 'tibble', 'universals', 'tibblify', 'styler', 'sessioninfo', 'tikzDevice', 'pillar', 'duckplyr', 'winch', 'withr', 'memoise', 'wrswoR', 'lest', 'drake', 'MakefileR', 'optparse', 'procmaps', 'traudem', 'utf8', 'dplyr', 'kimisc', 'extras', 'plogr', 'covr', 'profile', 'bindr', 'RPostgres', 'whereami', 'blob', 'sf', 'mockr', 'igraph', 'rprojroot']}]

> Finished chain.


'Kirill Müller contributes to a total of 54 packages. Some of the packages he contributes to include RSQLite, here, RMariaDB, DBI, chk, roxygen2md, mlfit, StrucDiv, term, dm, DBItest, log4r, hms, skimr, bindrcpp, desc, indiedown, enc, knitrProgressBar, fledge, cli, tibble, universals, tibblify, styler, sessioninfo, tikzDevice, pillar, duckplyr, winch, withr, memoise, wrswoR, lest, drake, MakefileR, optparse, procmaps, traudem, utf8, dplyr, kimisc, extras, plogr, covr, profile, bindr, RPostgres, whereami, blob, sf, mockr, igraph, and rprojroot.'

In [10]:
chain.run("what package will you recommend for osm data processing using the description ")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Package)
WHERE p.description CONTAINS 'osm data processing'
RETURN p.name
Full Context:
[]

> Finished chain.


"I'm sorry, but without any provided information, I don't have a specific package to recommend for OSM data processing. However, there are several popular packages available for this purpose, such as Osmosis, osmium-tool, and GDAL. These packages offer various functionalities for working with OSM data, including data extraction, filtering, and conversion. It would be helpful to consider your specific requirements and the programming language you are using to determine the most suitable package for your needs."

In [13]:
chain.run("what dependency has the most package depending on it and the maintainer")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Package)-[:DEPENDS_ON]->(d:Package)
WITH d, COUNT(p) AS dependencyCount
ORDER BY dependencyCount DESC
LIMIT 1
MATCH (d)<-[:MAINTAINS]-(m:Person)
RETURN d.name AS dependency, m.name AS maintainer
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have access to the information about the dependency with the most packages depending on it and its maintainer."

In [46]:
chain.run("using description properties recommend an alternative package for dplyr")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Package)-[:DEPENDS_ON]->(d:Dependency)<-[:DEPENDS_ON]-(p2:Package)
WHERE p1.package = 'dplyr'
RETURN p2.package, p2.description
Full Context:
[{'p2.package': 'dials', 'p2.description': 'Many models contain tuning parameters (i.e. parameters that\n    cannot be directly estimated from the data). These tools can be used\n    to define objects for creating, simulating, or validating values for\n    such parameters.'}, {'p2.package': 'googledrive', 'p2.description': 'Manage Google Drive files from R.'}, {'p2.package': 'retroharmonize', 'p2.description': 'Assist in reproducible retrospective (ex-post) harmonization\n    of data, particularly individual level survey data, by providing tools\n    for organizing metadata, standardizing the coding of variables, and\n    variable names and value labels, including missing values, and\n    documenting the data transformations, with the help of comprehensive\n    s3 classes.'}

'Based on the description properties, an alternative package for dplyr could be "dbplyr". It is a dplyr back end for databases that allows you to work with remote database tables as if they are in-memory data frames. While basic features work with any database that has a DBI back end, more advanced features require SQL translation to be provided by the package author.'

In [24]:
def evaluate_accuracy(chain, questions, expected_answers):
    num_correct = 0
    total_questions = len(questions)
    
    for i, question in enumerate(questions):
        generated_answer = chain.run(question)
        expected_answer = expected_answers[i]
        
        if expected_answer in generated_answer:
            num_correct += 1
            
    accuracy = num_correct / total_questions
    return accuracy

questions = [
    "How many packages is maintained by Kirill Müller?",
    "How many packages has Kirill Müller contributed to?",
    "Name of package Dennis Irorere contributed to",
    "Who maintains anyflights package",
    "Who is maintaining the most packages?",
    "Who is contributing to the most to packages?",
    'What package have the most contributions',
    "What packages depend on the most?"
]

expected_answers = [
    "28",
    "54",
    "anyflights",
    "Simon P. Couch",
    "Dirk Eddelbuettel",
    "Hadley Wickham",
    "mlpack",
    "stats"
]

In [25]:
accuracy = evaluate_accuracy(chain, questions , expected_answers)
print("Accuracy:", accuracy)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Kirill Müller'})-[:MAINTAINS]->(p:Package)
RETURN COUNT(p) as numberOfPackagesMaintained;
Full Context:
[{'numberOfPackagesMaintained': 28}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Kirill Müller'})-[:CONTRIBUTED_TO]->(p:Package)
RETURN COUNT(p) as num_packages_contributed
Full Context:
[{'num_packages_contributed': 54}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: "Dennis Irorere"})-[:CONTRIBUTED_TO]->(p:Package)
RETURN p.name
Full Context:
[{'p.name': 'anyflights'}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Package {name: "anyflights"})<-[:MAINTAINS]-(person:Person)
RETURN person;
Full Context:
[{'person': {'name': 'Simon P. Couch'}}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person

In [ ]:
num_correct

In [2]:
questions = [
    "How many packages is maintained by Kirill Müller?",
    "How many packages has Kirill Müller contributed to?",
    "Name of package Dennis Irorere contributed to",
    "Who maintains anyflights package",
    "Who is maintaining the most packages?",
    "Who is contributing to the most to packages?",
    "What are the packages that are supported by the most?",
    "What packages depend on the most?"
]


In [21]:
chain.run('What package is depend on the most?')



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Package)-[:DEPENDS_ON]->(p:Package)
RETURN p.name, COUNT(*) AS dependency_count
ORDER BY dependency_count DESC
LIMIT 1;
Full Context:
[{'p.name': 'stats', 'dependency_count': 5488}]

> Finished chain.


"The package that is depend on the most is 'stats' with a dependency count of 5488."

In [13]:
chain.run('Name of package Dennis Irorere contributed to')



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Dennis Irorere"})-[:CONTRIBUTED_TO]->(pkg:Package)
RETURN pkg.name
Full Context:
[{'pkg.name': 'anyflights'}]

> Finished chain.


'Dennis Irorere contributed to the package named anyflights.'

In [15]:
a = chain.run('Who maintains anyflights package')



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:MAINTAINS]->(pkg:Package {name: "anyflights"})
RETURN p
Full Context:
[{'p': {'name': 'Simon P. Couch'}}]

> Finished chain.


In [16]:
a

'Simon P. Couch maintains the anyflights package.'

In [23]:
accuracy = evaluate_accuracy(questions , expected_answers)
print("Accuracy:", accuracy)

TypeError: evaluate_accuracy() takes 2 positional arguments but 3 were given